In [1]:
import os
from dotenv import load_dotenv, find_dotenv
# Load environment variables from .env file
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
# Get the API key from environment variables
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

# Check if the API key is available
if not deepseek_api_key:
    raise ValueError("API key not found in environment variables.")

In [3]:
baichuan_api_key=os.getenv("BAICHUAN_AUTH_TOKEN")
if not baichuan_api_key:
    raise ValueError("API key not found in environment variables.")

In [4]:
# Import PyPDFLoader from langchain_community
from langchain_community.document_loaders import PyPDFLoader

# Specify the path to the PDF file
file_path = "book/1.pdf"

# Create a PyPDFLoader instance
loader = PyPDFLoader(file_path)
# Load the PDF document
docs = loader.load()
# Print the number of pages in the document
print(len(docs))

/Users/xuchen/Desktop/Project-ChatPDF-LLM-Development-with-LangChain/.qna/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1


In [5]:
# Print the first 100 characters of the content from the first page
print(docs[0].page_content[0:100])
# Print the metadata of the first page
print(docs[0].metadata)

第一回 桃园结义
  　　东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与
他的弟弟张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起
{'source': 'book/1.pdf', 'page': 0}


In [6]:
# Import ChatOpenAI from langchain_openai
from langchain_openai import ChatOpenAI
# Initialize a ChatOpenAI instance with deepseek model
llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_key=deepseek_api_key,
    openai_api_base="https://api.deepseek.com",
    max_tokens=1024
    )

In [7]:
from langchain_chroma import Chroma
from langchain_community.embeddings import BaichuanTextEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Using a text splitter, split the loaded documents into smaller documents that can more easily fit into an LLM's context window,
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# then load them into a vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=BaichuanTextEmbeddings(baichuan_api_key=baichuan_api_key))
# then create a retriever from the vector store for use in our RAG chain
retriever = vectorstore.as_retriever()

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "桃园三结义都有谁?"})

results

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'input': '桃园三结义都有谁?',
 'context': [Document(metadata={'page': 0, 'source': 'book/1.pdf'}, page_content='第一回 桃园结义\n  \u3000\u3000东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与\n他的弟弟张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。\n\u3000\u3000没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向官府，声势非常浩大。汉灵帝得到各地报告，连忙\n下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领一精一兵，分路攻打张角兄弟的黄巾军。\n\u3000\u3000张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招\n募兵马。\n\u3000\u3000榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进\n城来看榜文。\n\u3000\u3000刘备看完了榜文，不觉感慨地长叹了一声。忽听身后有个人大声喝道∶“大丈夫不给国家出力，叹什么\n气？”\n\u3000\u3000刘备回头一看，这人身高八尺，豹子头，圆眼睛，满腮的胡须像钢丝一样竖着，声音像洪钟，样子十分威\n武。那人对刘备说他姓张名飞，字翼德，做着卖酒、屠宰猪羊的生意。他愿意拿出家产作本钱，与刘备共同干\n一番大事业。\n\u3000\u3000刘备、张飞两人谈得投机，便一起到村 口的一家酒 店饮酒叙话。\n\u3000\u3000这时，一推车大汉进店饮酒。刘备 留神一看，这人有 九尺高，胸前长须飘飘，脸色好像红枣一样，长一 双\n丹凤眼，两条卧蚕眉，相貌非常威武雄 伟。刘备连忙起身， 邀他过来同坐，并请问姓名。\n\u3000\u3000那人说∶“ 我姓关名羽，字云长，因乡里恶霸仗势欺人，我一怒杀了恶霸，逃到外乡避难已有五、六年\n了。”刘备、张飞听了 都很敬佩，也将自己的志愿告诉了关羽。关羽听了也非常高兴。\n\u3000\u3000酒后他 们一同来到张飞的 庄上，只见庄后有一座桃园，园中桃 花灿烂，景色很美。第二天，三人在园中 焚\n香礼拜，宣誓结为异姓兄弟，三人 按年岁认了兄弟，刘备做了大 哥，关羽第二，张飞

In [9]:
print(results["context"][0].page_content)

第一回 桃园结义
  　　东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与
他的弟弟张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。
　　没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向官府，声势非常浩大。汉灵帝得到各地报告，连忙
下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领一精一兵，分路攻打张角兄弟的黄巾军。
　　张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招
募兵马。
　　榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进
城来看榜文。
　　刘备看完了榜文，不觉感慨地长叹了一声。忽听身后有个人大声喝道∶“大丈夫不给国家出力，叹什么
气？”
　　刘备回头一看，这人身高八尺，豹子头，圆眼睛，满腮的胡须像钢丝一样竖着，声音像洪钟，样子十分威
武。那人对刘备说他姓张名飞，字翼德，做着卖酒、屠宰猪羊的生意。他愿意拿出家产作本钱，与刘备共同干
一番大事业。
　　刘备、张飞两人谈得投机，便一起到村 口的一家酒 店饮酒叙话。
　　这时，一推车大汉进店饮酒。刘备 留神一看，这人有 九尺高，胸前长须飘飘，脸色好像红枣一样，长一 双
丹凤眼，两条卧蚕眉，相貌非常威武雄 伟。刘备连忙起身， 邀他过来同坐，并请问姓名。
　　那人说∶“ 我姓关名羽，字云长，因乡里恶霸仗势欺人，我一怒杀了恶霸，逃到外乡避难已有五、六年
了。”刘备、张飞听了 都很敬佩，也将自己的志愿告诉了关羽。关羽听了也非常高兴。
　　酒后他 们一同来到张飞的 庄上，只见庄后有一座桃园，园中桃 花灿烂，景色很美。第二天，三人在园中 焚
香礼拜，宣誓结为异姓兄弟，三人 按年岁认了兄弟，刘备做了大 哥，关羽第二，张飞最小，做了弟弟。
　　三人 请来铁匠打造兵器。刘备打 造了双股剑，关羽打了把八十二斤的青龙偃月刀，张飞造了一支丈八点钢
矛，各人又 造了一身铠甲。
　　他们聚集乡中壮士五百多人，浩浩 荡荡到涿郡去应募。
　　三人在涿 郡打败了黄巾军将领 程远志。刘备听说他 从前的老师中郎将卢植在 广宗和张角作战，便领了本 部
人马到广宗助战。卢植令刘备三兄弟 前往颍川帮助官军作战。刘备、 关羽、张飞引军连 夜奔赴颍川。


In [10]:
print(results["context"][0].metadata)

{'page': 0, 'source': 'book/1.pdf'}


resource: https://python.langchain.com/docs/tutorials/pdf_qa/